In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

In [3]:
# Check GPU using
# tf.config.experimental.list_physical_devices()
# tf.config.experimental.get_visible_devices()

# gpu = tf.config.experimental.list_physical_devices('XLA_GPU')[0]
# tf.config.experimental.set_memory_growth(gpu, True)

# Natural Language Processing
## Live Demos

Adapted from https://keras.io/examples/nlp/lstm_seq2seq/

### Download the data

In [4]:
# !!curl -O http://www.manythings.org/anki/fra-eng.zip
# !!unzip fra-eng.zip

### Configuration

In [5]:
BATCH_SIZE = 64
EPOCHS = 100
LATENT_DIM = 256
NUM_SAMPLES = 10000
DATA_PATH = "fra.txt"

### Prepare the data

In [6]:
# Vectorize data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(DATA_PATH, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

for line in lines[: min(NUM_SAMPLES, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # Tab is used as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text +"\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
            
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [7]:
print(f"Number of samples: {len(input_texts)}")
print(f"Number of unique input tokens: {num_encoder_tokens}")
print(f"Number of unique output tokens: {num_decoder_tokens}")
print(f"Max sequence length for inputs: {max_encoder_seq_length}")
print(f"Max sequence length for outputs: {max_decoder_seq_length}")

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [8]:
input_token_index = dict([(char,i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char,i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [9]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t+1:, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character
            decoder_target_data[i, t-1, target_token_index[char]] = 1.0
    decoder_input_data[i, t+1:, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

### Build the model

In [10]:
# Define the input sequence and process it
encoder_inputs = tf.keras.Input(shape=(None, num_encoder_tokens))
encoder = tf.keras.layers.LSTM(LATENT_DIM, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

In [11]:
# Discard encoder_outputs and only keep the states
encoder_states = [state_h, state_c]

# Set up the decoder, using 'encoder states' as initial state
decoder_inputs = tf.keras.Input(shape=(None, num_decoder_tokens))

In [12]:
# Set up our decoder to return full output sequences
# and to return internal states as well. The returns states
# are not used in the training model, but will be used in the inference.
decoder_lstm = tf.keras.layers.LSTM(LATENT_DIM, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = tf.keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

In [13]:
# Define the model that will turn `encoder_input_data` & 'decoder_input_data'
# into 'decoder_target_data'
model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

### Train the model

In [ ]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.2
)

In [ ]:
# Save model
model.save("s2s")

### Run inference (sampling)

In [15]:
# Define sampling models
# Restore the model and construct the encoder and decoder
model = tf.keras.models.load_model("s2s")

encoder_inputs = model.input[0] # input 1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output # lstm 1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = tf.keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1] # input 2
decoder_state_input_h = tf.keras.Input(shape=(LATENT_DIM,), name="input_3a")
decoder_state_input_c = tf.keras.Input(shape=(LATENT_DIM,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]

decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = tf.keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [16]:
# Reverse lookup token index to decode sequences back to something readable
reverse_input_char_index = dict((i,char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i,char) for char, i in target_token_index.items())

In [31]:
def decode_sequence(input_seq):
    # Encode the input as state vectors
    states_value = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character
    target_seq[0, 0, target_token_index["\t"]] = 1.0
    
    # Sampling loop for a batch of sequence
    # (to simplify, here we assume a batch size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        
        # Exit condition: either hit max length
        # or find stop character
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True
            
        # Update the target sequence (of length 1)
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0
        
        # Update states
        states_value = [h,c]
    
    return decoded_sentence

In [32]:
for seq_index in range(20):
    # Take one squence (part of the training set)
    # for trying out decoding
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sequence = decode_sequence(input_seq)
    print("-")
    print(f"Input sequence: {input_texts[seq_index]}")
    print(f"Decoded sequence: {decoded_sequence}")

-
Input sequence: Go.
Decoded sequence: Va.

-
Input sequence: Hi.
Decoded sequence: Sait-ile !

-
Input sequence: Hi.
Decoded sequence: Sait-ile !

-
Input sequence: Run!
Decoded sequence: Cours-tu ?

-
Input sequence: Run!
Decoded sequence: Cours-tu ?

-
Input sequence: Who?
Decoded sequence: Qui ?

-
Input sequence: Wow!
Decoded sequence: Ça alors !

-
Input sequence: Fire!
Decoded sequence: Au feu !

-
Input sequence: Help!
Decoded sequence: À l'aide !

-
Input sequence: Jump.
Decoded sequence: Saute.

-
Input sequence: Stop!
Decoded sequence: Arrête-toi !

-
Input sequence: Stop!
Decoded sequence: Arrête-toi !

-
Input sequence: Stop!
Decoded sequence: Arrête-toi !

-
Input sequence: Wait!
Decoded sequence: Attends !

-
Input sequence: Wait!
Decoded sequence: Attends !

-
Input sequence: Go on.
Decoded sequence: Poursuis.

-
Input sequence: Go on.
Decoded sequence: Poursuis.

-
Input sequence: Go on.
Decoded sequence: Poursuis.

-
Input sequence: Hello!
Decoded sequence: Salut !

